# Quickstart for California Housing Regression Model Documentation — Full Suite

This interactive notebook guides you through the process of documenting a model with the ValidMind Developer Framework. It uses the [California Housing Price Prediction](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.fetch_california_housing.html) sample dataset from Sklearn to train a simple regression model.

As part of the notebook, you will learn how to train a sample model while exploring how the documentation process works:

- Initializing the ValidMind Developer Framework
- Loading a sample dataset provided by the library to train a simple regression model
- Running a ValidMind test suite to quickly generate documention about the data and model

## ValidMind at a glance

ValidMind's platform enables organizations to identify, document, and manage model risks for all types of models, including AI/ML models, LLMs, and statistical models. As a model developer, you use the ValidMind Developer Framework to automate documentation and validation tests, and then use the ValidMind AI Risk Platform UI to collaborate on documentation projects. Together, these products simplify model risk management, facilitate compliance with regulations and institutional standards, and enhance collaboration between yourself and model validators.

If this is your first time trying out ValidMind, you can make use of the following resources alongside this notebook:

- [Get started](https://docs.validmind.ai/guide/get-started.html) — The basics, including key concepts, and how our products work
- [Get started with the ValidMind Developer Framework](https://docs.validmind.ai/guide/get-started-developer-framework.html) —  The path for developers, more code samples, and our developer reference

## Before you begin

::: {.callout-tip}
### New to ValidMind? 
For access to all features available in this notebook, create a free ValidMind account. 

Signing up is FREE — [**Sign up now**](https://app.prod.validmind.ai)
:::

If you encounter errors due to missing modules in your Python environment, install the modules with `pip install`, and then re-run the notebook. For more help, refer to [Installing Python Modules](https://docs.python.org/3/installing/index.html).

## Install the client library

The client library provides Python support for the ValidMind Developer Framework. To install it:

In [1]:
%pip install -q validmind


[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


## Initialize the client library

Every documentation project in the Platform UI comes with a _code snippet_ that lets the client library associate your documentation and tests with the right project on the Platform UI when you run this notebook. As you will see later, documentation projects are useful because they act as containers for model documentation and validation reports and they enable you to organize all of your documentation work in one place. 

Get your code snippet by creating a documentation project:

1. In a browser, log into the [Platform UI](https://app.prod.validmind.ai).

2. Go to **Documentation Projects** and click **Create new project**.

3. Select **`[Demo] Customer Churn Model`** and **`Initial Validation`** for the model name and type, give the project a unique  name to make it yours, and then click **Create project**.

4. Go to **Documentation Projects** > **YOUR_UNIQUE_PROJECT_NAME** > **Getting Started** and click **Copy snippet to clipboard**.

Next, replace this placeholder with your own code snippet:

In [2]:
## Replace this placeholder with the code snippet from your own project ##

import validmind as vm

vm.init(
  api_host = "http://localhost:3000/api/v1/tracking",
  api_key = "...",
  api_secret = "...",
  project = "..."
)
  

2024-01-19 14:08:27,288 - INFO(validmind.api_client): Connected to ValidMind. Project: [Demo] Customer Churn Model - Initial Validation (clr4sqr8z003muqy64hksfbnx)


## Initialize the Python environment

Next, let's import the necessary libraries and set up your Python environment for data analysis:

In [3]:
import pandas as pd
import sklearn
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

%matplotlib inline

## Load the sample dataset

To be able to use a sample dataset, you need to import the dataset and load it into a pandas [DataFrame](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html), a two-dimensional tabular data structure that makes use of rows and columns:

### Initialize a ValidMind dataset object

Before you can run a test suite, which are just a collection of tests, you must first initialize a ValidMind dataset object using the [`init_dataset`](https://docs.validmind.ai/validmind/validmind.html#init_dataset) function from the ValidMind (`vm`) module. 

This function takes a number of arguments: 

- `dataset` — the raw dataset that you want to analyze
- `target_column` — the name of the target column in the dataset 

In [4]:
from validmind.datasets.regression import california_housing as demo_dataset
df = demo_dataset.load_data()

In [5]:
vm_dataset = vm.init_dataset(
    dataset=df,
    target_column=demo_dataset.target_column,
)

2024-01-19 14:08:27,523 - INFO(validmind.client): Pandas dataset detected. Initializing VM Dataset instance...


In [6]:
full_suite = vm.run_test_suite(
    "tabular_dataset",
    inputs = {"dataset":vm_dataset},
)

2024-01-19 14:08:28,127 - ERROR(validmind.vm_models.test_suite.test): Failed to run test 'class_imbalance': (SkipTestError) Skipping ClassImbalance test astarget column as more than 10 classes


## Document the model

As part of documenting the model with the ValidMind Developer Framework, you need to preprocess the raw dataset, initialize some training and test datasets, initialize a model object you can use for testing, and then run the full suite of tests. 

### Prepocess the raw dataset

Preprocessing performs a number of operations to get ready for the subsequent steps:

- Preprocess the data: Splits the DataFrame (`df`) into multiple datasets (`train_df`, `validation_df`, and `test_df`) using `demo_dataset.preprocess` to simplify preprocessing.
- Separate features and targets: Drops the target column to create feature sets (`x_train`, `x_val`) and target sets (`y_train`, `y_val`).
- Initialize RandomForestRegressor regressor: Creates an `RandomForestRegressor` object with random state set to 0.
- Set evaluation metrics: Specifies metrics for model evaluation as "errors" and "r2".
- Fit the model: Trains the model on `x_train` and `y_train` using the validation set `(x_val, y_val)`. Verbose output is disabled.

In [7]:
train_df, validation_df, test_df = demo_dataset.preprocess(df)
x_train = train_df.drop(demo_dataset.target_column, axis=1)
y_train = train_df[demo_dataset.target_column]
x_val = validation_df.drop(demo_dataset.target_column, axis=1)
y_val = validation_df[demo_dataset.target_column]

Here we create two regression models so that the performance of the model can be compared through ValidMind test suite.

In [8]:
scale = False
if scale:
    scaler = StandardScaler()
    x_train = scaler.fit_transform(x_train)
    x_val = scaler.fit_transform(x_val)

model = RandomForestRegressor(random_state=0)
model.fit(x_train, y_train)
s1 = model.score(x_train, y_train)
s2 = model.score(x_val, y_val)
print("R² of Support Vector Regressor on training set: {:.3f}".format(s1))
print("R² of Support Vector Regressor on test set: {:.3f}".format(s2))

model_1 = GradientBoostingRegressor(random_state=0, max_depth=4)
model_1.fit(x_train, y_train)
model1_s1 = model_1.score(x_train, y_train)
model1_s2 = model_1.score(x_val, y_val)
print(
    "R² of Support Gradient Boosting Regressor on training set: {:.3f}".format(model1_s1))
print("R² of Support Gradient Boosting Regressor on test set: {:.3f}".format(model1_s2))

R² of Support Vector Regressor on training set: 0.918
R² of Support Vector Regressor on test set: 0.410
R² of Support Gradient Boosting Regressor on training set: 0.512
R² of Support Gradient Boosting Regressor on test set: 0.423


### Initialize the training and test datasets

With the datasets ready, you can now initialize the training and test datasets (`train_df` and `test_df`) created earlier into their own dataset objects using [`vm.init_dataset()`](https://docs.validmind.ai/validmind/validmind.html#init_dataset):

In [9]:
vm_train_ds = vm.init_dataset(
    dataset=train_df,
    target_column=demo_dataset.target_column
)

vm_test_ds = vm.init_dataset(
    dataset=test_df,
    target_column=demo_dataset.target_column
)

2024-01-19 14:08:37,895 - INFO(validmind.client): Pandas dataset detected. Initializing VM Dataset instance...
2024-01-19 14:08:37,896 - INFO(validmind.client): Pandas dataset detected. Initializing VM Dataset instance...


### Initialize a model object 

Additionally, you need to initialize a ValidMind model objects (`vm_model` and `vm_model_1`) that can be passed to other functions for analysis and tests on the data. You simply intialize this model object with [`vm.init_model()`](https://docs.validmind.ai/validmind/validmind.html#init_model):

In [10]:
vm_model = vm.init_model(
    model,
    train_ds=vm_train_ds,
    test_ds=vm_test_ds,
)
vm_model_1 = vm.init_model(
    model_1,
    train_ds=vm_train_ds,
    test_ds=vm_test_ds,
)

/Users/juanvalidmind/Dev/github/validmind/developer-framework/.venv/lib/python3.10/site-packages/sklearn/base.py:465: UserWarning:

X does not have valid feature names, but RandomForestRegressor was fitted with feature names

/Users/juanvalidmind/Dev/github/validmind/developer-framework/.venv/lib/python3.10/site-packages/sklearn/base.py:465: UserWarning:

X does not have valid feature names, but RandomForestRegressor was fitted with feature names

/Users/juanvalidmind/Dev/github/validmind/developer-framework/.venv/lib/python3.10/site-packages/sklearn/base.py:465: UserWarning:

X does not have valid feature names, but GradientBoostingRegressor was fitted with feature names

/Users/juanvalidmind/Dev/github/validmind/developer-framework/.venv/lib/python3.10/site-packages/sklearn/base.py:465: UserWarning:

X does not have valid feature names, but GradientBoostingRegressor was fitted with feature names



### Run the full suite of tests

This is where it all comes together: you are now ready to run the documentation tests for the model as defined by the documentation template you looked at earlier. 

The [`vm.run_test_suite`](https://docs.validmind.ai/validmind/validmind.html#run_test_suite) function finds and runs every tests specified in the test suites and then uploads all the documentation and test artifacts that get generated to the ValidMind AI Risk Platform. 

The function takes two arguments:

- `dataset`: The data to be tested, specified as `vm_dataset`.
- `model`: The candidate model to be used for testing, specified as `vm_model`.
-`models`: The list of models that can be compare with candidate model.

The variable `full_suite` then holds the result of these tests.

In [12]:
full_suite_model_performance = vm.run_test_suite(
    "regression_full_suite",
    inputs = {
        "dataset":vm_dataset,
        "model":vm_model,
        "models":[vm_model_1]
    }
)

2024-01-19 14:10:11,632 - ERROR(validmind.vm_models.test_suite.test): Failed to run test 'class_imbalance': (SkipTestError) Skipping ClassImbalance test astarget column as more than 10 classes


## Next steps

You can look at the results of this test suite right in the notebook where you ran the code, as you would expect. But there is a better way: view the test results as part of your model documentation right in the ValidMind Platform UI: 

1. Log back into the [Platform UI](https://app.prod.validmind.ai) 

2. Go to **Documentation Projects** > **YOUR_UNIQUE_PROJECT_NAME** > **Documentation**.

3. Expand the following sections and take a look around:
   
   - **2. Data Preparation**
   - **3. Model Development**

What you can see now is a much more easily consumable version of the documentation, including the results of the tests you just performed, along with other parts of your documentation project that still need to be completed. There is a wealth of information that gets uploaded when you run the full test suite, so take a closer look around, especially at test results that might need attention (hint: some of the tests in **2.1 Data description** look like they need some attention).

If you want to learn more about where you are in the model documentation process, take a look at [How do I use the framework?](https://docs.validmind.ai/guide/get-started-developer-framework.html#how-do-i-use-the-framework).

